In [1]:
import sys
sys.path.append('../')

from src.m4_generator import M4Generator
from src.m4_data_loader import M4DataLoader
from src.m4_evaluation_loss_functions import *
from src.visualization import *
from src.evaluation import *
from src.m4_model import M4Model
from glob import glob

import numpy as np
import keras
import matplotlib.pyplot as plt

import tensorflow as tf

%matplotlib inline

Using TensorFlow backend.


# Full Evaluation And Comparsion
Evaluate all trained models aginst all error functions

In [2]:

diffAugmentation = DiffAugmentation()
x_augmentations = [diffAugmentation]
y_augmentations = [diffAugmentation]

data = {'Model':[], 'Training Loss Function':[], 'Epochs':[], 'LOOKBACK': [], 'Hidden Layer Size':[],
        'Features Number':[], 'T-MASE': [], 'T-MASE Naive': [], 'H-MASE': [], 'H-MASE Naive': [],
        'T-Imporvement %': [],'H-Imporvement %': [], 
       }

models_arch = ['1-LSTM', '2-LSTM', '3-LSTM']
loss_functions = [m4_mase] #, smape, rmse, mae]

MODEL_BASE_DIR = '../models/point_predictions'
TRAINING_DATA_PATH = '../Dataset/Train/Hourly-train.csv'
TEST_DATA_PATH = '../Dataset/Test/Hourly-test.csv'

for model_arc in models_arch:
    s = f'{MODEL_BASE_DIR}/{model_arc}/*/'
    models = glob(s)
    for model_path in models:
        
        ev_results = {}
        
        for loss in loss_functions:
            ev_results = load_and_evaluate_model(model_path, '../Dataset/Train/Hourly-train.csv', 
                                                 '../Dataset/Test/Hourly-test.csv', 
                                                 x_augmentations, y_augmentations, loss)
        data['Model'].append(model_arc)
        data['Training Loss Function'].append(ev_results['hyperparameters']['loss'])
        data['Epochs'].append(ev_results['hyperparameters']['epochs'])
        data['LOOKBACK'].append(ev_results['hyperparameters']['lookback'])
        data['Hidden Layer Size'].append(ev_results['hyperparameters']['hidden_layer_size'])
        #data['Dropout Ratio'].append(ev_results['hyperparameters']['dropout_ratio'])
        data['Features Number'].append(ev_results['hyperparameters']['features_number'])
        data['T-MASE'].append(ev_results['test_error'])
        data['T-MASE Naive'].append(ev_results['naive_test_error'])
        data['H-MASE'].append(ev_results['holdout_error'])
        data['H-MASE Naive'].append(ev_results['naive_holdout_error'])
        
        test_naive_improv = (ev_results["naive_test_error"]- ev_results["test_error"])*100/ev_results["naive_test_error"]
        #test_snaive_improv = (ev_results["snaive_test_error"]- ev_results["test_error"])*100/ev_results["snaive_test_error"]

        holdout_naive_improv = (ev_results["naive_holdout_error"]- ev_results["holdout_error"])*100/ev_results["naive_holdout_error"]
        #holdout_snaive_improv = (ev_results["snaive_holdout_error"]- ev_results["holdout_error"])*100/ev_results["snaive_test_error"]
        
        data['T-Imporvement %'].append(round(test_naive_improv,3))
        
        #data['Test Imporvement sNaive %'].append(round(test_snaive_improv,3))
        data['H-Imporvement %'].append(round(holdout_naive_improv, 3))
        #data['holdout Imporvement sNaive %'].append(round(holdout_snaive_improv,3))
        #data['SMAPE'].append(model_results['test_error'][1])
        #data['RMSE'].append(model_results['test_error'][2])
        #data['MAE'].append(model_results['test_error'][3])

#for k in data.keys():
#    print(k)
#    print(len(data[k]))

df = pd.DataFrame(data=data)

dfStyler = df.style.set_properties(**{'text-align': 'center'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])


Loaded model from disk
Missing key'pi_params'
Loaded model from disk
Missing key'pi_params'
Loaded model from disk
Missing key'pi_params'
Loaded model from disk
Missing key'pi_params'
Loaded model from disk
Loaded model from disk
Missing key'pi_params'
Loaded model from disk
Missing key'pi_params'
Loaded model from disk
Missing key'pi_params'
Loaded model from disk
Missing key'pi_params'
Loaded model from disk
Missing key'pi_params'
Loaded model from disk
Missing key'pi_params'
Loaded model from disk
Missing key'pi_params'
Loaded model from disk
Missing key'pi_params'


,Model,Training Loss Function,Epochs,LOOKBACK,Hidden Layer Size,Features Number,T-MASE,T-MASE Naive,H-MASE,H-MASE Naive,T-Imporvement %,H-Imporvement %
0,1-LSTM,smape,100,48,100,1,2.431,20.57,1.507,3.689,88.182,59.149
1,1-LSTM,mae,100,48,200,2,2.421,20.57,1.302,3.689,88.23,64.706
2,1-LSTM,mase,100,48,200,1,2.114,20.57,1.36,3.689,89.723,63.134
3,1-LSTM,mae,100,48,200,1,1.912,20.57,1.319,3.689,90.705,64.245
4,1-LSTM,smape,100,48,200,2,2.261,20.57,1.51,3.689,89.008,59.067
5,2-LSTM,smape,100,48,80,1,2.263,20.57,1.496,3.689,88.999,59.447
6,2-LSTM,mase,100,48,100,1,2.212,20.57,1.383,3.689,89.246,62.51
7,3-LSTM,smape,100,48,50,2,2.236,20.57,1.539,3.689,89.13,58.281
8,3-LSTM,smape,100,48,50,1,2.567,20.57,1.507,3.689,87.521,59.149
9,3-LSTM,mase,100,48,120,1,2.233,20.57,1.351,3.689,89.144,63.378


In [3]:
df.to_csv('../models/point_predictions/comparsion.csv', sep='\t')